<a href="https://colab.research.google.com/github/OH1107/aisa_edu/blob/master/ML/ML_3%EC%9D%BC%EC%B0%A8_20200820_%EA%B5%90%EC%B0%A8%EA%B2%80%EC%A6%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 교차검증
- 교차검증 의미
- 실습

## 방법 1. 홀드아웃
- 데이터 : 훈련데이터와 테스트데이터
  + 훈련데이터 : 훈련데이터 + 검증데이터 -> 테스트데이터

## 방법 2. K폴드 교차검증
- 데이터의 수가 적을 때 주로 사용
- Default 연산량 X K개 만큼 연산됨 -> 시간 늘어남

- (cross_val_score)

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.datasets import load_boston

In [4]:
%matplotlib inline
boston = load_boston()

# pandas dataframe 독립변수
bostonDF = pd.DataFrame(boston.data, columns=boston.feature_names)

# 종속변수
bostonDF['PRICE'] = boston.target
print('Size of Dataset', bostonDF.shape)

Size of Dataset (506, 14)


In [5]:
bostonDF.head()

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT,PRICE
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98,24.0
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14,21.6
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03,34.7
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94,33.4
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33,36.2


### 데이터 탐색
- 시각화, 통계 검정
  + 시각화는 각자 연습
- 변수 정의
  + CRIM 마을 별 1인당 범죄율

  + ZN 25,000 평방 피트이상의 부지에 대해 구역된 주거용 토지의 비율

  + INDUS 도시당 비 소매업 에이커의 비율

  + CHAS Charles River에 대한 더미변수 (강의 경계에 위치한 경우는 1, 그렇지 않으면 0)

  + NOX 10ppm 당 농축 일산화질소

  + RM 주택 1가구당 평균 방의 개수

  + AGE 1940년 이전에 건축된 소유주택의 비율

  + DIS 5개의 보스턴 직업센터까지의 접근성 지수

  + RAD 방사형 도로까지의 접근성 지수

  + TAX $10,000 당 재산세율

  + PTRATIO 자치시별 학생/교사 비율

  + B 1000(Bk - 0.63)^2, 여기서 Bk는 자치시별 흑인의 비율을 말함

  + LSTAT % 모집단의 하위계층의 비율(%)

  + MEDV 본인 소유의주택가격(중앙값) (단위 : $1000)

## 머신러닝 모형 만들기

In [33]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

# 훈련데이터 , 테스트 데이터 분리
y_target = bostonDF['PRICE']
X_data = bostonDF.drop(['PRICE'], axis=1, inplace=False) # 현재 X_data에는 PRICE 컬럼만 떨어진 상태 , 점점 칼럼들을 드랍하면서 RMSE값에 대한 오차도를 줄이는 회귀검정을 이용

X_train, X_test, y_train, y_test = train_test_split(X_data, y_target, test_size=0.25, random_state=1)

In [34]:
y_target

0      24.0
1      21.6
2      34.7
3      33.4
4      36.2
       ... 
501    22.4
502    20.6
503    23.9
504    22.0
505    11.9
Name: PRICE, Length: 506, dtype: float64

In [35]:
X_data

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
0,0.00632,18.0,2.31,0.0,0.538,6.575,65.2,4.0900,1.0,296.0,15.3,396.90,4.98
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0,242.0,17.8,396.90,9.14
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0,242.0,17.8,392.83,4.03
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0,222.0,18.7,394.63,2.94
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0,222.0,18.7,396.90,5.33
...,...,...,...,...,...,...,...,...,...,...,...,...,...
501,0.06263,0.0,11.93,0.0,0.573,6.593,69.1,2.4786,1.0,273.0,21.0,391.99,9.67
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
503,0.06076,0.0,11.93,0.0,0.573,6.976,91.0,2.1675,1.0,273.0,21.0,396.90,5.64
504,0.10959,0.0,11.93,0.0,0.573,6.794,89.3,2.3889,1.0,273.0,21.0,393.45,6.48


In [36]:
X_test

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
307,0.04932,33.0,2.18,0.0,0.472,6.849,70.3,3.1827,7.0,222.0,18.4,396.90,7.53
343,0.02543,55.0,3.78,0.0,0.484,6.696,56.4,5.7321,5.0,370.0,17.6,396.90,7.18
47,0.22927,0.0,6.91,0.0,0.448,6.030,85.5,5.6894,3.0,233.0,17.9,392.74,18.80
67,0.05789,12.5,6.07,0.0,0.409,5.878,21.4,6.4980,4.0,345.0,18.9,396.21,8.10
362,3.67822,0.0,18.10,0.0,0.770,5.362,96.2,2.1036,24.0,666.0,20.2,380.79,10.19
...,...,...,...,...,...,...,...,...,...,...,...,...,...
41,0.12744,0.0,6.91,0.0,0.448,6.770,2.9,5.7209,3.0,233.0,17.9,385.41,4.84
361,3.83684,0.0,18.10,0.0,0.770,6.251,91.1,2.2955,24.0,666.0,20.2,350.65,14.19
289,0.04297,52.5,5.32,0.0,0.405,6.565,22.9,7.3172,6.0,293.0,16.6,371.72,9.51
498,0.23912,0.0,9.69,0.0,0.585,6.019,65.3,2.4091,6.0,391.0,19.2,396.90,12.92


In [37]:
X_train

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD,TAX,PTRATIO,B,LSTAT
502,0.04527,0.0,11.93,0.0,0.573,6.120,76.7,2.2875,1.0,273.0,21.0,396.90,9.08
172,0.13914,0.0,4.05,0.0,0.510,5.572,88.5,2.5961,5.0,296.0,16.6,396.90,14.69
80,0.04113,25.0,4.86,0.0,0.426,6.727,33.5,5.4007,4.0,281.0,19.0,396.90,5.29
46,0.18836,0.0,6.91,0.0,0.448,5.786,33.3,5.1004,3.0,233.0,17.9,396.90,14.15
318,0.40202,0.0,9.90,0.0,0.544,6.382,67.2,3.5325,4.0,304.0,18.4,395.21,10.36
...,...,...,...,...,...,...,...,...,...,...,...,...,...
255,0.03548,80.0,3.64,0.0,0.392,5.876,19.1,9.2203,1.0,315.0,16.4,395.18,9.25
72,0.09164,0.0,10.81,0.0,0.413,6.065,7.8,5.2873,4.0,305.0,19.2,390.91,5.52
396,5.87205,0.0,18.10,0.0,0.693,6.405,96.0,1.6768,24.0,666.0,20.2,396.90,19.37
235,0.33045,0.0,6.20,0.0,0.507,6.086,61.5,3.6519,8.0,307.0,17.4,376.75,10.88


In [38]:
# 모형을 만들기 - 선형회귀
lr_model = LinearRegression()
lr_model.fit(X_train, y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [39]:
# 예측
y_preds = lr_model.predict(X_test)
y_preds

## sklearn은 RMSE 함수를 제공하지 않는다

array([32.37816533, 27.95684437, 18.07213891, 21.63166556, 18.93029508,
       19.96277202, 32.2834674 , 18.06715668, 24.72989076, 26.85359369,
       27.23326816, 28.57021239, 21.18778302, 26.94393815, 23.37892579,
       20.89176865, 17.11746934, 37.73997945, 30.51980066,  8.44489436,
       20.86557977, 16.21989418, 25.13605925, 24.77658813, 31.40497629,
       11.02741407, 13.82097563, 16.80208261, 35.94637198, 14.7155729 ,
       21.23939821, 14.15079469, 42.72492585, 17.83887162, 21.84610225,
       20.40178099, 17.50287927, 27.00093206,  9.80760408, 20.00288662,
       24.27066782, 21.06719021, 29.47089776, 16.48482565, 19.38852695,
       14.54778282, 39.39838319, 18.09810655, 26.22164983, 20.60676525,
       25.09994066, 24.48366723, 25.02297948, 26.84986898,  5.01517985,
       24.12809513, 10.72843392, 26.83178157, 16.8023533 , 35.48142073,
       19.50937911, 27.43260347, 16.58016763, 19.151488  , 10.9990262 ,
       32.05016535, 36.32672849, 21.8596379 , 24.8158357 , 25.32

- mean_squared_error(MSE) : 실제값과 예측값의 차이를 제곱 후 평균
- RMSE : 실제 오류 평균보다 더 커지는 특성이 있어서 루트 계산

In [40]:
# MSE 계산
mse = mean_squared_error(y_test, y_preds)
mse

21.897765396049422

In [41]:
# RMSE (mse에 루트)
rmse = np.sqrt(mse)
rmse

4.679504823808757

## K 폴드 적용
- cross_val_score() 함수 활용해서 RMSE를 다시 구해보자
  + 5 폴드값, 값에 대한 평균

In [49]:
from sklearn.model_selection import cross_val_score

neg_mse_scores = cross_val_score(lr_model, X_data, y_target, scoring='neg_mean_squared_error', cv=5)

# scoring : 성능검증에 사용할 함수 이름
# 함수 종류 링크 참조 : https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter

neg_mse_scores

array([-12.46030057, -26.04862111, -33.07413798, -80.76237112,
       -33.31360656])

In [50]:
rmse_scores=np.sqrt(-1 * neg_mse_scores)
rmse_scores

array([3.52991509, 5.10378498, 5.75101191, 8.9867887 , 5.77179405])

In [51]:
avg_rmse = np.mean(rmse_scores)
avg_rmse

5.828658946215802